In [42]:
import requests
import json, re, time
from datetime import datetime
import pandas as pd

In [43]:
with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json',
            'cookie': 'ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip; ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version"
    ]

In [44]:
def getTestSettings(cc, url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc
                      }
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

In [ ]:
def getTestSettingsTraceroute(cc, url):
    json_test = {
                  "testSettings": {
                    "BufferSize": 32,
                    "Count": 3,
                    "Fragment": 1,
                    "Ipv4only": 0,
                    "Ipv6only": 0,
                    "MaxFailedHops": 0,
                    "Resolve": 1,
                    "Sleep": 300,
                    "Ttl": 128,
                    "TtlStart": 1,
                    "Timeout": 60000,
                    "HopTimeout": 1000,
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc
                      }
                    ],
                    "Destinations": [
                      url
                    ],
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

### StartPageLoadTest function

In [45]:
def startPageLoadTest(test_settings):
    
    test_url = API_ENDPOINT + "StartPageLoadTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartPageLoadTestResult']['Status']['StatusText']):
        return res['StartPageLoadTestResult']['TestID']
    else:
        return "FAILED"

### StartTracerouteTest function

In [58]:
def startTracerouteTest(test_settings):
    
    test_url = API_ENDPOINT + "StartTracertTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartTracerouteTestResult']['Status']['StatusText']):
        return res['StartTracerouteTestResult']['TestID']
    else:
        return "FAILED"

### retrievePageLoadTestResults function

In [46]:
def retrievePageLoadTestResults(testID):
    
    url = API_ENDPOINT + "GetPageLoadResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['PageLoadTestResults']

### retrieveTracerouteTestResults function

In [59]:
def retrieveTracerouteTestResults(testID):
    
    url = API_ENDPOINT + "GetTracertResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['TracerouteTestResults']

## Retrieve results

In [47]:
def getResultsFromTestIDs(df_tests_url):
    df_results = pd.DataFrame(columns=['domain',
                                       'cc',
                                       'ProbeInfo',
                                       'TestDateTime',
                                       'StatusCode',
                                       'StatusText',
                                       'DNSLookupTime', 
                                       'Destination', 
                                       'HTTPStatus', 
                                       'InitialConnection', 
                                       'NumberOfRequests', 
                                       'PageLoadTime', 
                                       'SSLNegotiationTime', 
                                       'TTFB', 
                                       'TotalDownloadedBytes'
                                      ])
    
    #Exclude FAILED items
    for index, row in df_tests_url.loc[df_tests_url.testID!='FAILED'].iterrows():
        
        domain = row['domain']
        cc = row['cc']
        testID = row['testID']
        pageLoadTestResults =  retrievePageLoadTestResults(testID)

        if (pageLoadTestResults is None or len(pageLoadTestResults) < 1):
            continue
        
        for res in pageLoadTestResults:
        
            probeInfo = res['ProbeInfo']
            testDateTime = res['TestDateTime']

            #extract the epoch
            m = re.search('/Date\((\d+)\+0000\)/', testDateTime)
            if m:
                testDateTime = m.group(1)[:-3]

            #convert into human readable format
            testDateTime = datetime.fromtimestamp(int(testDateTime)).strftime('%Y-%m-%d')
            statusCode = res['TestStatus']['StatusCode']
            statusText = res['TestStatus']['StatusText']
            dnsLookupTime = res['DNSLookupTime']
            destination =  res['Destination']
            hTTPStatus = res['HTTPStatus']
            initialConnection = res['InitialConnection']
            numberOfRequests =  res['NumberOfRequests'] 
            pageLoadTime =  res['PageLoadTime'] 
            sslNegotiationTime = res['SSLNegotiationTime'] 
            ttfb = res['TTFB']
            totalDownloadedBytes = res['TotalDownloadedBytes']

            df_results = df_results.append({
                'domain': domain,
                'cc': cc,
                'ProbeInfo':probeInfo,
                'TestDateTime': testDateTime,
                'StatusCode':statusCode,
                'StatusText':statusText,
                'DNSLookupTime':dnsLookupTime,
                'Destination':destination,
                'HTTPStatus':hTTPStatus,
                'InitialConnection':initialConnection,
                'NumberOfRequests':numberOfRequests,
                'PageLoadTime':pageLoadTime,
                'SSLNegotiationTime':sslNegotiationTime,
                'TTFB':ttfb,
                'TotalDownloadedBytes':totalDownloadedBytes
            }, ignore_index=True)

    return df_results

In [48]:
def runMeasurements(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettings(cc, url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    
    

In [49]:
df_links = pd.read_csv('data/links_sample.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [ ]:
runMeasurements(df_links, 'url', 'data/url_testID.csv')
#runMeasurements(df_links, 'ori_amp_url', 'data/ori_amp_url_testID.csv')
#runMeasurements(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID.csv')
#runMeasurements(df_links, 'amp_cdn_url', 'data/amp_cdn_url_testID.csv')

In [51]:
df_testIDs_url = pd.read_csv('data/url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_ori_amp = pd.read_csv('data/ori_amp_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_viewer = pd.read_csv('data/amp_viewer_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_cdn = pd.read_csv('data/amp_cdn_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [52]:
#df_results_ori_amp = getResultsFromTestIDs(df_testIDs_ori_amp)
#df_results_amp_viewer = getResultsFromTestIDs(df_testIDs_amp_viewer)
#df_results_amp_cdn = getResultsFromTestIDs(df_testIDs_amp_cdn)
df_results_url = getResultsFromTestIDs(df_testIDs_url)

In [53]:
df_results_url.to_csv("data/url_results" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_ori_amp.to_csv("data/ori_amp_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_viewer.to_csv("data/amp_viewer_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_cdn.to_csv("data/amp_cdn_url_results.csv", sep='|', encoding='utf-8', index=False)

In [55]:
df_results_url = df_results_url[(df_results_url.HTTPStatus=='200') & (df_results_url.StatusText=='OK')]

In [57]:
df_results_url.to_csv("data/url_results.csv", sep='|', encoding='utf-8', index=False)
